In [1]:
from moviepy.editor import *
from moviepy.video.fx.all import fadein, loop
from conversions import add_music
import random
import math

class Generator:
    def __init__(self, video, music, overlay='assets/grainoverlay'):
        self.video = video
        self.music = music
        self.overlay = overlay
        
    def randomShuffle(self):
        newVideo = self.video.subclip(3, math.floor(self.video.duration) - 10)
        currLen = 0
        vidClip = []
        while currLen < newVideo.duration:
            if newVideo.duration - currLen <= 8:
                vidClip.append(newVideo.subclip(currLen, currLen + (newVideo.duration - currLen)))
                break
            else:
                randomNum = random.randint(3,4)
                tempClip = newVideo.subclip(currLen, currLen + randomNum)
                if randomNum % 2 == 0:
                    tempClip = fadein(tempClip, 3)
                vidClip.append(tempClip)
                currLen += randomNum
        for index in range(len(vidClip)):
            randIndex = random.randint(0, len(vidClip) - 1)
            vidClip[index], vidClip[randIndex] = vidClip[randIndex], vidClip[index]
        newVideo.close()
        return concatenate_videoclips(vidClip).set_duration(self.music.duration)
    
    def generateOverlay(self):
        overlay = self.overlay.subclip().resize(self.video.size).set_opacity(0.50)
        
        if overlay.duration < self.music.duration:
            overlay = loop(overlay, duration=self.music.duration)
        else:
            overlay = overlay.set_duration(self.music.duration)
        return overlay 

    def create(self):
        final = CompositeVideoClip([self.randomShuffle(), self.generateOverlay()])
        final.write_videofile('testOutput.mp4', audio=False)
        add_music('testOutput.mp4','travis.mp3')

In [1]:
from scipy.io.wavfile import read, write
from scipy import signal
import numpy as np
import subprocess
from conversions import convert_wav, convert_mp3, find_name

class Reverb:
    
    def __init__(self, musicLocation, filterLocation='impulses/French 18th Century Salon.wav'):
        self.music_location = musicLocation
        self.filter_location = filterLocation
        self.music = None
        self.filter = None
        
    def import_music(self):
        audio = read(self.music_location)
        audio_arr = np.array(audio[1], dtype='float')
        self.music = audio_arr
        
    def import_filter(self):
        impulse = read(self.filter_location)
        impulse_arr = np.array(impulse[1], dtype='float')
        impulse_arr = np.multiply(impulse_arr, 1.0/np.max(impulse_arr))
        self.filter = impulse_arr
    
    def convert_reverb(self):
        self.import_music()
        self.import_filter()
        filtered = signal.convolve(self.music, self.filter, mode='same', method='fft')
        filtered = np.multiply(filtered, 1.0/np.max(np.abs(filtered)))
        return filtered
    
    def export(self):
        final = self.convert_reverb()
        musicName = find_name(self.music_location)
        write(musicName+'.wav', rate=40000, data=final)
        convert_mp3(musicName, 'slowed_reverbed')

In [42]:
convert_wav('test/edm')

In [45]:
Reverb('test/edm.wav').export()

In [14]:
convert_wav('test/edm.mp3')

In [2]:
convert_wav('crashing')

NameError: name 'convert_wav' is not defined

In [3]:
from reverb import Reverb

In [34]:
Reverb('johnmayer.wav').export()

In [4]:
convert_wav('crashing')

NameError: name 'convert_wav' is not defined

In [40]:
Reverb('bazzi.wav').export()

In [39]:
convert_wav('bazzi')

In [1]:
from reverb import Reverb
from conversions import convert_wav

In [2]:
convert_wav('latenight')

In [3]:
Reverb('latenight.wav').export()